In [1]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import lib.utils as utils
from lib.constants import S, B, C, INPUT_SIZE, CLASS_NAMES, CLASS_COLORS
import os
import cv2
import pickle

In [2]:
def convert_bounding_boxes(bounding_boxes):
    out_boxes = []
    for box in bounding_boxes:
        box_dict = None
        b_x = (int)((box[0][0]+box[1][0])/2) / 2
        b_y = (int)((box[0][1]+box[1][1])/2) / 2
        b_w = (int)(abs(box[0][0]-box[1][0])) / 2
        b_h = (int)(abs(box[0][1]-box[1][1])) / 2
        box_dict = {'b_x':b_x,
                    'b_y':b_y,
                    'b_w':b_w,
                    'b_h':b_h,
                    'c':box[2]}
        out_boxes.append(box_dict)
    return out_boxes


In [3]:
start_point = (None, None)
end_point = (None, None)


def callback(event, x, y, flags, param):
    global start_point, end_point, clone_image, draw_image, color

    if event == cv2.EVENT_LBUTTONDOWN:
        if start_point == (None, None):
            start_point = (x, y)
        else:
            end_point = (x, y)
            bounding_boxes.append([start_point, end_point, class_name])
            cv2.rectangle(clone_image, start_point, (x, y), color=color)
            #clone_image = utils.createIndependetCopy(draw_image)
            start_point = (None, None)
            end_point = (None, None)

    if event == cv2.EVENT_MOUSEMOVE:
        if start_point != (None, None):
            draw_image = utils.createIndependetCopy(clone_image)
            cv2.putText(draw_image, class_name, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.rectangle(draw_image, start_point, (x, y), color=color)


In [4]:
image_names = os.listdir('./../resources/train_data/')
print(image_names)

#cv2.namedWindow('img')
#cv2.setMouseCallback('img', callback)

colors = dict(zip(CLASS_NAMES, CLASS_COLORS))

for image_name in image_names:

    bounding_boxes = []

    image = utils.loadImage(path='./../resources/train_data/', name=image_name, resize=(INPUT_SIZE, INPUT_SIZE), color=False)
    name = image_name.replace('.png','')

    original_image = cv2.cvtColor(cv2.resize(image, (2*INPUT_SIZE,2*INPUT_SIZE)), cv2.COLOR_GRAY2BGR)
    clone_image = utils.createIndependetCopy(original_image)
    draw_image = utils.createIndependetCopy(original_image)

    cv2.namedWindow('img')
    cv2.setMouseCallback('img', callback)

    for class_name in CLASS_NAMES:
        color = colors[class_name]

        draw_image = utils.createIndependetCopy(clone_image)
        cv2.putText(draw_image, class_name, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    
        while(1):

            #clone_image = utils.createIndependetCopy(original_image)
            cv2.imshow('img', draw_image)

            k = cv2.waitKey(10)

            if k == ord('z'):
                bounding_boxes = bounding_boxes[:-1]
                clone_image = utils.createIndependetCopy(original_image)
                for box in bounding_boxes:
                    cv2.rectangle(clone_image, box[0], box[1], color=colors[box[2]])
                draw_image = utils.createIndependetCopy(clone_image)
                cv2.putText(draw_image, class_name, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

            #space
            if k == 32:
                break

            #escape
            if k == 27:
                cv2.destroyAllWindows()
                sys.exit()

    size = (int)(INPUT_SIZE/S)
    for x in range(0, S):
        for y in range(0, 2*INPUT_SIZE):
            draw_image[y][x*size*2] = (0,0,0)


    for y in range(0, S):
        for x in range(0, 2*INPUT_SIZE):
            draw_image[y*size*2][x] = (0,0,0)

    for box in bounding_boxes:
        center_x = (int)((box[0][0]+box[1][0])/2)
        center_y = (int)((box[0][1]+box[1][1])/2)
        cv2.circle(draw_image, (center_x,center_y),6,colors[box[2]],-1)
    

    while(cv2.waitKey(10) != ord('n')):
        cv2.imshow('img', draw_image)


    utils.saveImage(name=image_name, image=image, path='./../resources/labeled_data/images/', color=False)

    bounding_boxes = convert_bounding_boxes(bounding_boxes)

    label = open(str('./../resources/labeled_data/labels/' + name + '.txt'), 'wb')
    pickle.dump(bounding_boxes, label)
    label.close()


cv2.destroyAllWindows()

['10.png', '11.png', '8.png', '9.png']


In [5]:




#    for i in range(0, len(data)):
#        image = convertImg(utils.loadImage(path='./../resources/trainData/{class_name}'.format(class_name=class_name), name=data[i], resize=(32,32)))
#        if(i/len(data) < ratio):
#            train_data.append((image,class_names.index(class_name)))
#        else:
#            test_data.append((image,class_names.index(class_name)))
#
#
#for x in range(0, n_x):
#    for y in range(0, n_y):
#        section = image[y*size:(y*size)+size, x*size:(x*size)+size]
#        cv2.imshow('img', cv2.cvtColor(cv2.resize(section, (128,128)), cv2.COLOR_RGB2BGR))
#        cv2.waitKey(0)
#
#cv2.destroyAllWindows()
# mouse callback function
#def draw_circle(event,x,y,flags,param):
#    if event == cv2.EVENT_LBUTTONDBLCLK:
#        cv2.circle(img,(x,y),100,(255,0,0),-1)
## Create a black image, a window and bind the function to window
#
#img = np.zeros((512,512,3), np.uint8)
#cv2.namedWindow('image')
#cv2.setMouseCallback('image',draw_circle)
#while(1):
#    cv2.imshow('image',img)
#    if cv2.waitKey(20) & 0xFF == 27:
#        break
#cv2.destroyAllWindows()